<a href="https://colab.research.google.com/github/Rohith-Bollam/new-project/blob/main/PromptEDU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade google-generativeai langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
API_KEY=userdata.get('GOOGLE_API_KEY_1')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)

In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [5]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 4.4 MB/s eta 0:00:00


In [7]:
from langchain_community.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader('/content/22365_3_Prompt Engineering_v7 (1).pdf')
loaded_pdf_doc = pdfloader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50, separators=['\n\n', '\n', ' '])

In [9]:
chunks = recursive_splitter.split_documents(loaded_pdf_doc)
print(len(chunks))

351


In [10]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 7.0 MB/s eta 

In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001',google_api_key=API_KEY)

In [13]:
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(documents=chunks, embedding=embeddings)

In [14]:
vector_index = vector_store.as_retriever(search_kwargs={'k':8})

In [15]:
from langchain.chains import RetrievalQA
rag = RetrievalQA.from_chain_type(
    model,
    retriever = vector_index,
    return_source_documents=True
)

In [18]:
question = 'what are the chain of thoughts according to the doc?'
response = rag({"query":question})
print(response)

{'query': 'what are the chain of thoughts according to the doc?', 'result': 'Based on the document, here are key aspects of Chain of Thought (CoT):\n\n*   **Definition**: Chain of thought is a prompting technique where the LLM generates a sequence of intermediate reasoning steps to solve a problem.\n*   **Use Cases**: It\'s useful for tasks that can be solved by "talking through" the steps, such as code generation or breaking down a complex task. If you can explain the steps to solve a problem, CoT is a good candidate.\n*   **Implementation**: CoT can be implemented as "zero-shot" (without examples) or with single-shot or few-shot examples.\n*   **Benefits**: The document mentions that Chain of Thought prompting can be very powerful when combined with a single-shot or few-shot examples.\n*   **Relation to Tree of Thoughts**: Chain of thought is a linear sequence of thoughts, whereas Tree of Thoughts allows for branching out and exploring different reasoning paths.', 'source_documents':